In [18]:
#Загрузим необходимые библиотеки

import pandas as pd

from yargy import Parser, rule, or_
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy import interpretation as interp
from yargy.relations import gnc_relation

gnc = gnc_relation()

In [22]:
#Загрузим и посмотрим датасет

df = pd.read_csv('test_data.csv')
df.sample(10)

,dlg_id,line_n,role,text
365,5,28,manager,По поводу тарифов лицензии отдельно виджет отд...
156,1,47,manager,А куда вам можно на ватсап например
110,1,1,manager,Алло здравствуйте
343,5,6,manager,Так да да хорошо смотрите давайте по порядку
192,2,28,client,Так значит у вас пишет и все
243,2,79,client,Угу хорошо
387,5,50,manager,У вас получится всего в сумме 13 месяцев там о...
142,1,33,client,У меня кстати также 5 5 пользователей
349,5,12,manager,Угу
204,2,40,client,А сделать обязательно несколько раз


In [23]:
#Создадим парсер для извлечения приветствия

Greeting = fact(
    'Greeting',
    ['phrase']
)

greeting = morph_pipeline([
    'Привет', 'Здравствуй', 'Добрый вечер', 'Добрый день', 'Доброе утро'
]).interpretation(
    interp.normalized()
)

GREETING = rule(greeting).interpretation(
            Greeting.phrase
        )


pars_greet = Parser(GREETING)

In [24]:
#Создадим функцию для проверки наличия приветствия

def pars_greeting(text):
    gr = list(pars_greet.findall(text))
    if gr:
        return True
    else:
        return False

In [25]:
#Добавим колонку с репликами приветствия

df['greeting'] = df.apply(
    lambda x: x.text
    if x.role == 'manager' and pars_greeting(x.text) else None,
    axis=1)

In [26]:
#Создадим парсер для извлечения имени менеджера

Name = fact(
    'Name',
    ['npro', 'verb', 'name']
)

FULL_NAME = or_(
    rule(
        gram('NPRO').interpretation(
            Name.npro), 
        gram('VERB').interpretation(
            Name.verb), 
        gram('Name').interpretation(
            Name.name.inflected()
        ).match(gnc)
    ),
    rule(
        gram('NPRO').interpretation(
            Name.npro), 
        gram('Name').interpretation(
            Name.name.inflected()
        ).match(gnc), 
        gram('VERB').interpretation(
            Name.verb)
    )
).interpretation(Name)

    
pars_name = Parser(FULL_NAME)

In [28]:
#Создадим функцию для извлечения имени менеджера или всей реплики представления

def pars_names(text, is_=0):
    names = list(pars_name.findall(text))
    if names and is_ == 0:
        return names[0].fact.name.title()
    elif names and is_ == 1:
        return names[0].fact.npro + ' ' + names[0].fact.verb + ' ' + names[
            0].fact.name.title()
    else:
        return None

In [29]:
#Создадим колонку с репликой представления

df['is_name'] = df.apply(
    lambda x: pars_names(x.text, is_=1)
    if x.role == 'manager' and pars_names(x.text, is_=1) != None else None,
    axis=1)

In [30]:
#Создадим колонку с именем менеджера

df['name'] = df.apply(lambda x: pars_names(x.text)
                      if x.role == 'manager' else None,
                      axis=1)

In [31]:
#Создадим парсер для извлечения названия компании

Company = fact(
    'Company',
    ['type', 'one', 'two']
)

COMPANY = morph_pipeline([
    'Компания', 'Фирма', 'Предприятие'
]).interpretation(
    Company.type.normalized()
)

COMPANY_RULE = or_(
    rule(
        COMPANY, 
        gram('Name').interpretation(
            Company.one), 
        gram('NOUN').interpretation(
            Company.two).optional()
    ),
    rule(
        COMPANY,
        gram('NOUN').interpretation(
            Company.one)
    )
).interpretation(Company)

pars_comp = Parser(COMPANY_RULE)

In [32]:
#Создадим функцию для извлечения названия компании

def pars_company(text):
    comps = list(pars_comp.findall(text))
    if comps:
        if comps[0].fact.two:
            return comps[0].fact.type.title() + ' ' + comps[0].fact.one.title(
            ) + ' ' + comps[0].fact.two
        else:
            return comps[0].fact.type.title() + ' ' + comps[0].fact.one.title()
    else:
        return None

In [33]:
#Создадим колонку с названием компании

df['company'] = df.apply(lambda x: pars_company(x.text)
                         if x.role == 'manager' else None,
                         axis=1)

In [34]:
#Создадим парсер для извлечения реплики прощания

Parting = fact(
    'Parting',
    ['phrase']
)

parting = morph_pipeline([
    'Пока', 'До свидания', 'До встречи', 'Хорошего дня', 'Хорошего вечера', 
    'Прощай', 'Будьте здоровы', 'Всего доброго', 'Всего хорошего'
]).interpretation(
    interp.normalized()
)

PARTING = rule(parting).interpretation(
            Parting.phrase
        )


pars_part = Parser(PARTING)

In [35]:
#Создадим функцию для извлечения реплики прощания

def pars_parting(text):
    pr = list(pars_part.findall(text))
    if pr:
        return True
    else:
        return False

In [36]:
#Создадим колонку с репликой прощания

df['parting'] = df.apply(
    lambda x: x.text if x.role == 'manager' and pars_parting(x.text) else None,
    axis=1)

In [37]:
#Создадим колонку с проверкой обязательного требования к менеджеру

df['is_script'] = df.apply(lambda x: 'Yes' if sum(df[
    (df.role == 'manager') &
    (df.dlg_id == x.dlg_id)].agg(lambda x: sum(x.notna(
    )))[['greeting', 'parting']] != 0) == 2 and x.role == 'manager' else None,
                           axis=1)

In [40]:
#Посмотрим на получившийся результат

df

,dlg_id,line_n,role,text,greeting,is_name,name,company,parting,is_script
0,0,0,client,Алло,None,None,None,None,None,None
1,0,1,manager,Алло здравствуйте,Алло здравствуйте,None,None,None,None,Yes
2,0,2,client,Добрый день,None,None,None,None,None,None
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,None,Меня зовут Ангелина,Ангелина,Компания Диджитал бизнес,None,Yes
4,0,4,client,Ага,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,None,None,None,None,None,None
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,None,None,None,None,None,None
477,5,140,client,Спасибо спасибо,None,None,None,None,None,None
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,None,None,None,None,None,None
